# Forcasting  

In this notebook we will predict the closing prices for the next month

In [7]:
# Import Libraries
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

AttributeError: module 'matplotlib' has no attribute 'rcParams'

In [ ]:
# Load DataSet
data= pd.read_csv('./data/engineered_weekly_stock_market.csv')


data.head()

,companyName,date,openingPrice,highestPrice,lowestPrice,closingPrice,volume,year,month,day_of_month,week_of_year,price_range,price_change,weekly_return,log_volume,moving_avg_4,ema_4,volatility_4
0,AMEN BANK,2014-06-16,-0.104873,-0.105238,-0.105602,-0.105532,-0.111736,2014,6,16,25,-0.069502,-0.067207,-0.608059,-0.506038,-0.134304,-0.105530,-0.129781
1,AMEN BANK,2014-06-23,-0.105491,-0.105851,-0.106085,-0.106023,-0.105668,2014,6,23,26,-0.077417,-0.056514,-0.504264,0.336594,-0.134304,-0.105727,-0.129781
2,AMEN BANK,2014-06-30,-0.105983,-0.106064,-0.106466,-0.106136,-0.099024,2014,6,30,27,-0.068063,-0.024434,-0.159442,0.583623,-0.134304,-0.105891,-0.129781
3,AMEN BANK,2014-07-07,-0.105983,-0.105814,-0.106708,-0.106036,-0.111046,2014,7,7,28,-0.039998,-0.015879,-0.065430,-0.243828,-0.105453,-0.105948,-0.106254
4,AMEN BANK,2014-07-14,-0.105870,-0.105826,-0.105818,-0.106023,-0.110070,2014,7,14,29,-0.091090,-0.024434,-0.159025,-0.046500,-0.105576,-0.105978,-0.125022


In [ ]:
data= data.drop(columns=['companyName','date'])


In [ ]:
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    data = np.array(data)

    for i in range(len(data)-seq_length):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        sequences.append(x)
        labels.append(y)

    return np.array(sequences), np.array(labels)

seq_length = 5
X, y = create_sequences(data, seq_length)

print("Sequences")
print(X)
print("Labels")
print(y)

Sequences
[[[-0.10487338 -0.10523785 -0.10560218 ... -0.13430394 -0.10552998
   -0.12978145]
  [-0.10549142 -0.10585124 -0.10608507 ... -0.13430394 -0.10572694
   -0.12978145]
  [-0.10598333 -0.10606405 -0.1064663  ... -0.13430394 -0.10589056
   -0.12978145]
  [-0.10598333 -0.10581369 -0.10670774 ... -0.1054531  -0.10594834
   -0.10625432]
  [-0.10586981 -0.1058262  -0.10581821 ... -0.10557636 -0.10597795
   -0.12502186]]

 [[-0.10549142 -0.10585124 -0.10608507 ... -0.13430394 -0.10572694
   -0.12978145]
  [-0.10598333 -0.10606405 -0.1064663  ... -0.13430394 -0.10589056
   -0.12978145]
  [-0.10598333 -0.10581369 -0.10670774 ... -0.1054531  -0.10594834
   -0.10625432]
  [-0.10586981 -0.1058262  -0.10581821 ... -0.10557636 -0.10597795
   -0.12502186]
  [-0.10598333 -0.10590131 -0.10593258 ... -0.10557636 -0.10599572
   -0.12502186]]

 [[-0.10598333 -0.10606405 -0.1064663  ... -0.13430394 -0.10589056
   -0.12978145]
  [-0.10598333 -0.10581369 -0.10670774 ... -0.1054531  -0.10594834
   -0.

In [ ]:
X = X.astype(np.float32)
y = y.astype(np.float32)

In [ ]:
# Split into train and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, X.shape[2])))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
966/966 [==============================] - 4s 3ms/step - loss: 247464.3281 - val_loss: 243457.5781
Epoch 2/20
966/966 [==============================] - 3s 3ms/step - loss: 241177.5000 - val_loss: 239564.2500
Epoch 3/20
966/966 [==============================] - 3s 3ms/step - loss: 238746.1719 - val_loss: 238294.9062
Epoch 4/20
966/966 [==============================] - 3s 3ms/step - loss: 238100.7188 - val_loss: 238073.1094
Epoch 5/20
966/966 [==============================] - 3s 3ms/step - loss: 238018.5156 - val_loss: 238060.8906
Epoch 6/20
966/966 [==============================] - 3s 3ms/step - loss: 238015.4531 - val_loss: 238060.7812
Epoch 7/20
966/966 [==============================] - 3s 3ms/step - loss: 238015.4531 - val_loss: 238060.7812
Epoch 8/20
966/966 [==============================] - 3s 3ms/step - loss: 238015.4375 - val_loss: 238060.8125
Epoch 9/20
966/966 [==============================] - 3s 3ms/step - loss: 238015.4219 - val_loss: 238060.8125
Epoch 10/2

In [ ]:
# Reshape the predicted values
y_pred = y_pred.reshape(y_test.shape)

# Calculate evaluation metrics using reshaped values
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')

242/242 [==============================] - 0s 861us/step


ValueError: y_true and y_pred have different number of output (16!=1)